<a href="https://colab.research.google.com/github/mzohaibnasir/GenAI/blob/main/09_EndtoEndMedicalChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-End Medical Chatbot